In [ ]:
# This chunk of code make it possible to use src functions
import sys
import os
module_path = os.path.abspath(os.path.join('../src'))

if module_path not in sys.path:
    sys.path.append(module_path)
import warnings
warnings.filterwarnings('ignore')
    
from dsproject import dsproject
import pandas as pd
from model import *

dsp = dsproject()

In [ ]:
from sklearn.preprocessing import StandardScaler
import numpy as np
train = dsp.read_table(train_dataset + '_group1_prediction', 'output')
train.loc[train['Percentage']>1] = 1.0
train = train.loc[train['group1_prediction']==0]

In [ ]:
def preprocess_percentage(df, scaler=None):
    if not scaler:
        scaler = StandardScaler().fit(df['Percentage'])
    df['Percentage_trans'] = scaler.transform(df['Percentage'])
    return df, scaler

In [ ]:
#train, scaler = preprocess_percentage(train)

%matplotlib inline
import seaborn as sns
sns.distplot(train['Percentage'])

In [ ]:
validate = dsp.read_table(validate_dataset + '_group5_prediction', 'output')
validate.loc[validate['Percentage']>1] = 1.0
#validate, scaler = preprocess_percentage(validate, scaler=scaler)
validate = validate.loc[validate['group1_prediction']==0]

In [ ]:
score = dsp.read_table(score_dataset + '_group5_prediction', 'output')
score.loc[score['Percentage']>1] = 1.0

In [ ]:
x, y, col_to_drop_numeric, col_to_drop_cat = feature_selection(train, 'Percentage', func_numeric=f_regression, func_cat=f_regression)
x_validate, y_validate, _, _ = feature_selection(validate, 'Percentage', col_to_drop_numeric, col_to_drop_cat, func_numeric=f_regression, func_cat=f_regression)

In [ ]:
from sklearn.ensemble import RandomForestRegressor as RFR
from sklearn.linear_model import LinearRegression as LR
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

def test_model(model, x_train, x_test, y_train, y_test):
    model.fit(x_train, y_train)
    yhat = model.predict(x_test)
    result = sqrt(mean_squared_error(y_test, yhat))
    print("Error of prediction : %2.3f" % result)
    return model

def train_regressor(x, y):
    estim = {'LR': LR(n_jobs=-1), 
             'RFR': RFR(n_estimators=50, n_jobs=-1, oob_score=True, min_samples_leaf=10, random_state=42)}
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

    mod = {}
    for key, value in estim.items():
        mod[key] = test_model(value, x_train, x_test, y_train, y_test)
    return mod, (x_train, x_test, y_train, y_test)

In [ ]:
def validate_model(model, x, y):
    yhat = model.predict(x_test)
    result = sqrt(mean_squared_error(y_test, yhat))
    print("Error of prediction : %2.3f" % result)
    return result

In [ ]:
modr1, (x_train, x_test, y_train, y_test) = train_regressor(x, y)

In [ ]:
validate_model(modr1['RFR'], x_validate, y_validate)

Customer values

In [ ]:
yhat = modr1['RFR'].predict(x_validate)

In [ ]:
%matplotlib inline
sns.distplot(yhat)

In [ ]:
sns.regplot(yhat, validate['Percentage'])

In [ ]:
from sklearn.metrics import r2_score
r2_score(validate['Percentage'], yhat)

In [ ]:
from scipy.stats import spearmanr, pearsonr
spearmanr(validate['Percentage'], yhat)

In [ ]:
pearsonr(validate['Percentage'], yhat)